# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import cv2
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load Data

In [ ]:
train_df = pd.read_csv("/media/alper/B00A1E730A1E36B6/Datasets/MURA-v1.1/train_image_paths.csv", header=None, names=["image_path"])
train_df.head()

In [ ]:
train_df["label"] = train_df["image_path"].map(lambda x: "positive" if "positive" in x else "negative")
train_df.head()

In [ ]:
train_df.label.value_counts()

In [ ]:
train_df["category"] = train_df["image_path"].apply(lambda x: x.split("/")[2])
train_df.head()

In [ ]:
train_df.category.value_counts()

In [ ]:
train_df["patientId"] = train_df["image_path"].apply(lambda x: x.split("/")[3].replace("patient", ""))
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.shape

In [ ]:
train_df.isnull().sum()

In [ ]:
valid_df = pd.read_csv("/media/alper/B00A1E730A1E36B6/Datasets/MURA-v1.1/valid_image_paths.csv", header=None, names=["image_path"])
valid_df.head()

In [ ]:
valid_df["label"] = valid_df["image_path"].map(lambda x: "positive" if "positive" in x else "negative")
valid_df.head()

In [ ]:
valid_df.label.value_counts()

In [ ]:
valid_df["category"] = valid_df["image_path"].apply(lambda x: x.split("/")[2])
valid_df.head()

In [ ]:
valid_df.category.value_counts()

In [ ]:
valid_df["patientId"] = valid_df["image_path"].apply(lambda x: x.split("/")[3].replace("patient", ""))
valid_df.head()

In [ ]:
valid_df.shape

In [ ]:
valid_df.info()

In [ ]:
valid_df.isnull().sum()

In [ ]:
train_df = train_df[train_df["category"] == "XR_WRIST"]
valid_df = valid_df[valid_df["category"] == "XR_WRIST"]

In [ ]:
train_df.shape

In [ ]:
valid_df.shape

In [ ]:
wrist_pos = glob.glob("/media/alper/B00A1E730A1E36B6/Datasets/MURA-v1.1/train/XR_WRIST/patient00012/study1_negative/*")
wrist_neg = glob.glob("/media/alper/B00A1E730A1E36B6/Datasets/MURA-v1.1/train/XR_WRIST/patient00006/study1_positive/*")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 7))
ax = ax.ravel()

for i in range(0, 3):
    ax[i].imshow(plt.imread(wrist_pos[i]), cmap="gray")
    ax[i].set_title("XR_WRIST POSITIVE")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 7))
ax = ax.ravel()

for i in range(0, 3):
    ax[i].imshow(plt.imread(wrist_neg[i]), cmap="gray")
    ax[i].set_title("XR_WRIST NEGATIVE")

In [ ]:
train_data = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rotation_range=45,
    fill_mode="nearest"
)

valid_data = ImageDataGenerator(
    rescale=1./255
)

In [ ]:
train_gen = train_data.flow_from_dataframe(
    dataframe=train_df,
    directory="/media/alper/B00A1E730A1E36B6/Datasets/",
    x_col="image_path",
    y_col="label",
    batch_size=32,
    shufle=True,
    class_mode="categorical",
    target_size=(128, 128),
    save_to_dir="train",
    save_format="png",
    color_mode="grayscale"
)

valid_gen = valid_data.flow_from_dataframe(
    dataframe=valid_df,
    directory="/media/alper/B00A1E730A1E36B6/Datasets/",
    x_col="image_path",
    y_col="label",
    batch_size=32,
    shufle=True,
    class_mode="categorical",
    target_size=(128, 128),
    save_to_dir="valid",
    save_format="png",
    color_mode="grayscale"
)

# Model

In [ ]:
base_model = MobileNetV2(weights=None, include_top=False, input_shape=(128, 128, 1))

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
predictions = Dense(2, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(
    train_gen, 
    steps_per_epoch=train_gen.samples // train_gen.batch_size, 
    epochs=10, 
    validation_data=valid_gen, 
    validation_steps=valid_gen.samples // valid_gen.batch_size,
)

In [ ]:
print(history.history)

In [ ]:
plt.figure()
plt.title("Model Accuracy")
plt.plot(history.history["accuracy"], label="train")
plt.plot(history.history["val_accuracy"], label="valid")
plt.legend()
plt.show()

In [ ]:
plt.figure()
plt.title("Model Loss")
plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="valid")
plt.legend()
plt.show()

In [ ]:
model.save("mura.h5")